In [ ]:
from importlib import reload ,import_module
import module.utilize as utilize
import module.singleVariant as singleVariant
import module.multiVariant as multiVariant
import numpy as np
from numba import njit,jit, float32
from tqdm import tqdm
import module.multiHistogramSparse as multiHistogramSparse
import module.ananlysisFuncion as analysisFunction
import module.singleVariantCopulaBase as CopulaBase
reload(multiHistogramSparse)
reload(utilize)
reload(multiVariant)
reload(singleVariant)
reload(analysisFunction)
reload(CopulaBase)

#attribute_names=np.array(["SALT","TEMP","U","V","W"])
attribute_names=np.array(["phi_grav","particle_mass_density","zmom","ymom","xmom"])
attribute_names=np.array(["phi_grav","particle_mass_density","zmom","ymom","xmom"])
incremental_number=300
all_ensamble_data=np.empty([0,incremental_number,64,64,64])

for name in attribute_names:
    data=utilize.readFiles(name,incremental_number)
    data=data.reshape(1,incremental_number,64,64,64)
    all_ensamble_data=np.append(all_ensamble_data,data,axis=0)



covBlockSize=5
dataBlockSize=5
binsNumber=128
sizeZ=60
sizeY=60
sizeX=60
minMaxBlockSize=2
isMinMax=False

conditions=np.array([[36,40],[26,30]])

print("start fit model")
with tqdm(total=3, desc="Model fitting") as pbar:
    #oursModel=multiVariant.multiDistCopula3D(all_ensamble_data,dataBlockSize,covBlockSize,binsNumber,[sizeZ,sizeY,sizeX],minMaxBlockSize,isMinMax)
    oursModel=multiVariant.multiDistCopula3D.load(f"Nyx_{attribute_names.shape[0]}varaibles_{incremental_number}members_128Bins_dBlock5_cBlock5_new")
    #conditions=np.array([[0,1e5],[3e10,5e10]])
    #oursModel.fit()
    print("ours complete fit")
    pbar.update(1)
    copulaBlockSize=2
    copulaBaseModel=CopulaBase.multiVariantCopulaBase(all_ensamble_data,copulaBlockSize)
    copulaBaseModel.fit()
    print("copula complete fit")
    pbar.update(1)
    multiHistBlockSize=2
    multiHistModel=multiHistogramSparse.multiHistogramSpaseModel(all_ensamble_data,blockSize=multiHistBlockSize,binsNumber=binsNumber)
    multiHistModel.fit()
    print("multi-hist complete fit")
   
    print("complete fit")
    pbar.update(1)

start fit model


Model fitting:  33%|███▎      | 1/3 [00:00<00:00,  5.11it/s]

ours complete fit


Model fitting:  67%|██████▋   | 2/3 [01:26<00:50, 50.93s/it]

copula complete fit


Model fitting: 100%|██████████| 3/3 [02:34<00:00, 51.43s/it]

multi-hist complete fit
complete fit


In [9]:
oursModel.getStorageSize()
counts=np.array([17.2, 5.3, 3.9, 4.4,3.8])
copulaBaseModel.getStorageSize(counts)
multiHistModel.getStorageSize()

nonZeros=[]
for i in range(len(multiHistModel.blocks)):
    nonZeros.append(len(multiHistModel.blocks[i].hist))

nonZeros=np.array(nonZeros)
print(f"sparse non zeros counts:{nonZeros.mean()}")

v0: 357480
v1: 110016
v2: 80928
v3: 92268
v4: 80100
cov:36288.0
sum:757080.0
copula base storage:3742105.5999999996
sparse sum: 14398170
sparse non zeros counts:73.23287963867188
